# Testing Apache Spark queries 

Disclaimer: O intuito deste trabalho é meramente aprender a utilizar o Apache Spark. Embora os dados utilizados tenham teor político, não há intenção de fazer qualquer tipo apologia, seja positiva ou negativa, a qualquer partido ou candidato.

O autor do trabalho declara não ter vínculo algum com nenhum partido político ou candidato.

Com intuito de negar quaisquer questionamentos, tentarei usar algumas estratégias:
- Sempre utilizar funções aleatórias para printar os dados
- Tentar trabalhar o problema de forma geral, sem focar em nenhum partido ou candidato
- Evitar apresentar nomes de candidatos ao menos que seja necessário

## Instantiating Spark session

In [107]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from pyspark.sql.types import (
    StructType, 
    StructField, 
    StringType, 
    IntegerType, 
    DateType,
    FloatType
)

In [108]:
spark = SparkSession.builder.appName("Test_queries").getOrCreate()

/home/joaopedro/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


## Loading csv files from the folder

Bens declarados dos Candidatos: https://dados.gov.br/dataset/candidatos-2022

Brasil states: https://github.com/fititnt/gis-dataset-brasil

### Loading bens

In [109]:
df_bens_raw = (
    spark
    .read
    .option("header", "true")
    .option("encoding", 'latin1')
    .option("delimiter", ";")
    .option("inferSchema", "true")
    .csv('./bem_candidato_2022/bem_candidato_2022_BRASIL.csv')
)

Filtrando as colunas usadas e transformando o valor do bem em float

In [110]:
df_bens = (
    df_bens_raw
    .select(
        [
            "SQ_CANDIDATO",
            "DS_TIPO_BEM_CANDIDATO",
            "DS_BEM_CANDIDATO",
            "VR_BEM_CANDIDATO"
        ]
    )
    .withColumn(
        'VR_BEM_CANDIDATO',
        F.regexp_replace('VR_BEM_CANDIDATO', ',', '.')
        .cast(FloatType())
    )
)

In [111]:
df_bens.show(5)

+------------+---------------------+--------------------+----------------+
|SQ_CANDIDATO|DS_TIPO_BEM_CANDIDATO|    DS_BEM_CANDIDATO|VR_BEM_CANDIDATO|
+------------+---------------------+--------------------+----------------+
|110001608768| Veículo automotor...|               Carro|         40000.0|
|200001608811| Fundos: Ações, Mú...|5.617,50 AÇÕES DA...|        50164.28|
|240001614377|              Terreno|TERRENO URBANO,  ...|        186200.0|
|240001614377| Caderneta de poup...|CADERNETA DE POUP...|         2367.39|
|210001647159| Depósito bancário...|SALDO APLICACAO F...|        22713.17|
+------------+---------------------+--------------------+----------------+
only showing top 5 rows



### Loading Candidatos

In [112]:
df_candidatos_raw = (
    spark
    .read
    .option("header", "true")
    .option("encoding", 'latin1')
    .option("delimiter", ";")
    .option("inferSchema", "true")
    .csv('./consulta_cand_2022/consulta_cand_2022_BRASIL.csv')
)

In [113]:
df_deputados_fed = (
    df_candidatos_raw
    .filter(df_candidatos_raw['DS_SITUACAO_CANDIDATURA'] != 'INAPTO')
    .filter(df_candidatos_raw['CD_CARGO'] == '6') # Código do cargo de deputado federal
    .select(
        [
            "SG_UE",
            "NM_UE",
            "SQ_CANDIDATO",
            "NM_CANDIDATO",
            "SG_PARTIDO",
        ]
    )
)

Visualizando os dados ...

In [114]:
(
    df_deputados_fed
    .select( ["SG_UE", "SG_PARTIDO"] )
    .sample(False, 0.01)
    .show(10)
)

+-----+----------+
|SG_UE|SG_PARTIDO|
+-----+----------+
|   SC|       PSD|
|   MG|      NOVO|
|   MS|      PSDB|
|   SC|       PSD|
|   MG|       PMN|
|   MG|  PATRIOTA|
|   SP|        PP|
|   PE|       PSB|
|   AM|      PSDB|
|   RS|      PROS|
+-----+----------+
only showing top 10 rows



## Realizando o Join entre os dois DataFrames

Unir as duas tabelas usando o campo `SQ_CANDIDATO` como chave

In [115]:
df_candidatos_bens = (
    df_deputados_fed
    .join(
        df_bens, 
        on='SQ_CANDIDATO', 
        how='left'
    )
    .filter(
        F.col('VR_BEM_CANDIDATO').isNotNull()
    )
)

In [116]:
df_candidatos_bens.sample(False, 0.01).show(5)

+------------+-----+-----------------+--------------------+----------+---------------------+--------------------+----------------+
|SQ_CANDIDATO|SG_UE|            NM_UE|        NM_CANDIDATO|SG_PARTIDO|DS_TIPO_BEM_CANDIDATO|    DS_BEM_CANDIDATO|VR_BEM_CANDIDATO|
+------------+-----+-----------------+--------------------+----------+---------------------+--------------------+----------------+
|190001619508|   RJ|   RIO DE JANEIRO|FABIANO OLIVEIRA ...|     UNIÃO| Quotas ou quinhõe...|99% das cotas da ...|         19800.0|
|190001638606|   RJ|   RIO DE JANEIRO|WELDERSON SIDNEY ...|  PATRIOTA|                 Casa|no município de V...|         93000.0|
|210001613993|   RS|RIO GRANDE DO SUL|LUCIANO PALMA DE ...|       PSD|                 Loja|Rua Coronel Chicu...|        470172.6|
|190001619480|   RJ|   RIO DE JANEIRO|CLEBIO LOPES PEREIRA|     UNIÃO| OUTROS BENS E DIR...|            SIDERCOL|       1676989.6|
|160001622616|   PR|           PARANÁ|NELTON MIGUEL FRI...|       PDT| Aplicação de

# Queries

Existem duas formas principais de realizar queries no apache Spark aqui no Python: Funcional e via SQL.
Essa diferença é meramente de sintaxe e escolha pessoal, pois o Spark executa as queries da mesma forma.

Como eu tenho mais familiaridade com o Pandas, para mim, é mais natural utilizar o encadeamento de funções.

**Top 10 candidatos a Deputado Federal com maior patrimônio**

In [117]:
# Nomes e patrimonio dos Candidatos mais ricos
patrimonio_candidatos = (
    df_candidatos_bens
    .groupBy("SQ_CANDIDATO")
    .agg(
        F.sum("VR_BEM_CANDIDATO").alias("VR_BEM_CANDIDATO")
    )
    .join(
        df_deputados_fed,
        on='SQ_CANDIDATO',
        how='left'
    )
    .orderBy(
        F.desc("VR_BEM_CANDIDATO")
    )
    .select(
        [
            "NM_CANDIDATO",
            "SG_PARTIDO",
            "SG_UE",
            "VR_BEM_CANDIDATO",
        ]
    )
    .withColumn(
        "VR_BEM_CANDIDATO",
        F.format_number(
            F.col("VR_BEM_CANDIDATO") / 1e6, 2
        )
    )
)

patrimonio_candidatos.show(10, truncate=False)

+----------------------------------+----------+-----+----------------+
|NM_CANDIDATO                      |SG_PARTIDO|SG_UE|VR_BEM_CANDIDATO|
+----------------------------------+----------+-----+----------------+
|EUNÍCIO LOPES DE OLIVEIRA         |MDB       |CE   |158.18          |
|RUY ADRIANO BORGES MUNIZ          |AVANTE    |MG   |158.02          |
|JOSE GOMES FERREIRA FILHO         |PP        |DF   |128.64          |
|JADYEL SILVA ALENCAR              |PV        |PI   |107.55          |
|PABLO HENRIQUE COSTA MARÇAL       |PROS      |SP   |88.44           |
|ALEX DOS SANTOS GARCIA            |PSC       |RN   |80.01           |
|ADRIANA MANGABEIRA WANDERLEY      |PSD       |DF   |77.06           |
|NEWTON BONIN                      |UNIÃO     |PR   |76.12           |
|HERCILIO ARAUJO DINIZ FILHO       |MDB       |MG   |65.90           |
|PAULO ROBERTO ROQUE ANTONIO KHOURI|NOVO      |DF   |65.29           |
+----------------------------------+----------+-----+----------------+
only s

**Valor total declarado**

In [118]:
total_candidatos_bens = (
    df_candidatos_bens
    .agg(
        F.sum("VR_BEM_CANDIDATO").alias("TOTAL_BEM_CANDIDATO_BILHOES"),
        F.countDistinct("SQ_CANDIDATO").alias("TOTAL_CANDIDATOS")
    )
    .withColumn(
        "TOTAL_BEM_CANDIDATO_BILHOES",
        F.format_number(
            F.col("TOTAL_BEM_CANDIDATO_BILHOES") / 1e9, 2
        )
    )
)

In [119]:
total_candidatos_bens.collect()

[Row(TOTAL_BEM_CANDIDATO_BILHOES='7.21', TOTAL_CANDIDATOS=6366)]

**Valor total declarado de bens por UF**

In [120]:
valor_por_uf = (
    df_candidatos_bens
    .groupBy(
        "SG_UE"
    )
    .agg(
        F.sum( F.col("VR_BEM_CANDIDATO") ).alias("TOTAL BENS")
    )
    .orderBy(
        F.desc("TOTAL BENS")
    )
)

In [121]:
valor_por_uf.show(30, truncate=False)

+-----+--------------------+
|SG_UE|TOTAL BENS          |
+-----+--------------------+
|SP   |1.2143276856198936E9|
|MG   |9.504191711380489E8 |
|RJ   |5.998792442260875E8 |
|PR   |4.687364865754497E8 |
|DF   |4.3387237743134475E8|
|GO   |3.1266681836336124E8|
|CE   |2.976540537267532E8 |
|BA   |2.840689506582558E8 |
|RS   |2.6503172541893113E8|
|SC   |2.4177128341094604E8|
|MA   |2.0494777125577286E8|
|PE   |2.028025139128284E8 |
|PA   |2.0055216283283943E8|
|MT   |1.9994814255513492E8|
|RR   |1.7936015321644616E8|
|PI   |1.626300498293633E8 |
|RN   |1.5322524850030357E8|
|AC   |1.2491514657306153E8|
|ES   |1.2232408985722113E8|
|TO   |1.078568051314967E8 |
|MS   |1.0426919895911348E8|
|RO   |7.584124175289346E7 |
|PB   |6.894666267027906E7 |
|AL   |6.780417662602329E7 |
|AM   |5.897766750664139E7 |
|SE   |5.343360440528E7    |
|AP   |4.920965397933769E7 |
+-----+--------------------+



**Valor total declarado de bens por Tipo de bem**

In [122]:
from pyspark.sql.window import Window

In [123]:
valor_por_tipo_bem = (
    df_candidatos_bens
    .groupBy(
        "DS_TIPO_BEM_CANDIDATO"
    )
    .agg(
        F.sum( F.col("VR_BEM_CANDIDATO") ).alias("TOTAL BENS")
    )
    .orderBy(
        F.desc("TOTAL BENS")
    )
    .withColumn(
        "Percentual",
        # Trick to get the percentage of each row
        # https://stackoverflow.com/questions/48915834/how-to-calculate-percentage-of-each-row-in-pyspark
        F.col("TOTAL BENS") / F.sum(F.col("TOTAL BENS")).over(Window.partitionBy()) * 100
    )
)   

In [124]:
valor_por_tipo_bem.show(10, truncate=False)

+------------------------------------------------------------+--------------------+------------------+
|DS_TIPO_BEM_CANDIDATO                                       |TOTAL BENS          |Percentual        |
+------------------------------------------------------------+--------------------+------------------+
|Casa                                                        |1.0589040910269547E9|14.6958322559435  |
|Quotas ou quinhões de capital                               |9.503185969242871E8 |13.188845721200835|
|Apartamento                                                 |8.900710215289758E8 |12.352709314382224|
|Outros bens imóveis                                         |6.791679916829965E8 |9.425725109532399 |
|OUTROS BENS E DIREITOS                                      |5.823552953922777E8 |8.082125479509068 |
|Terreno                                                     |5.2137738439256346E8|7.235853225994051 |
|Outras participações societárias                            |4.374760876

## Saving answers to csv files

In [125]:
ANSWERS_PATH = './answers'

# Convert to Pandas
# and save to CSV

patrimonio_candidatos.toPandas().to_csv(
    f'{ANSWERS_PATH}/patrimonio_candidatos.csv',
    index=False
)

total_candidatos_bens.toPandas().to_csv(
    f'{ANSWERS_PATH}/total_candidatos_bens.csv',
    index=False
)

valor_por_uf.toPandas().to_csv(
    f'{ANSWERS_PATH}/valor_por_uf.csv',
    index=False
)

valor_por_tipo_bem.toPandas().to_csv(
    f'{ANSWERS_PATH}/valor_por_tipo_bem.csv',
    index=False
)


In [126]:
spark.stop()